In [1]:
!curl localhost:9200

{
  "name" : "404bd7aad9ec",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "yGvsFlj_TBmdpNC7S0G5pg",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [4]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
response = es.indices.create(index=index_name, body=index_settings)

response

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [6]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:22<00:00, 42.05it/s]


In [7]:
query = "How do I execute a command in a running docker container?"

In [10]:
def retrieve_documents(query, index_name="course-questions", max_results=5):
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    return response

In [11]:
response = retrieve_documents(query)
response['hits']['hits'][0]

{'_index': 'course-questions',
 '_id': 'ITbIQJABXWhjtG9BlPHv',
 '_score': 84.050095,
 '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'}}

In [12]:
response['hits']['hits'][0]['_score']

84.050095

In [13]:
def retrieve_documents(query, index_name="course-questions", max_results=5):
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [41]:
documents = retrieve_documents(query)
documents = documents[:3]
documents[-1]['question']

'How do I copy files from a different folder into docker container’s working directory?'

In [42]:
context_template = """
Question: {question}
Answer: {text}
""".strip()

context_docs = documents

context_result = ""

for doc in documents:
    doc_str = context_template.format(question=doc.get('question'),
                                      text=doc.get('text')
                                     )
    context_result += ("\n\n" + doc_str)

context = context_result.strip()

print(context)

Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Question: How do I copy files from my local machine to docker container?
Answer: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Question: How do I copy files from a different folder into docker container’s working directory?
Answer: You can copy files from your local machine into a Docker container using t

In [43]:
prompt_template = f"""
You're a course teaching assistant. Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database. 
Only use the facts from the CONTEXT. If the CONTEXT doesn't contan the answer, return "NONE"

QUESTION: {query}

CONTEXT:

{context}
""".strip()

In [44]:
prompt = prompt_template.format(question=query, context=context)

In [45]:
len(prompt)

1555

In [46]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [47]:
len(encoding.encode(prompt))

337

In [48]:
encoding.decode(encoding.encode(prompt))

'You\'re a course teaching assistant. Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database. \nOnly use the facts from the CONTEXT. If the CONTEXT doesn\'t contan the answer, return "NONE"\n\nQUESTION: How do I execute a command in a running docker container?\n\nCONTEXT:\n\nQuestion: How do I debug a docker container?\nAnswer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQuestion: How do I copy files from my local machine to docker container?\nAnswer: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker

In [49]:
import os

api_key = os.environ["MISTRAL_API_KEY"]

In [50]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

client = MistralClient(api_key=api_key)


def llm_chatbot(prompt):
    response = client.chat(
        model='open-mistral-7b',
        messages=[
            ChatMessage(role="user", content=prompt)
        ]
    )

    return response.choices[0].message.content

In [51]:
response = llm_chatbot(prompt)

In [55]:
print(response)

To execute a command in a running Docker container, you can use the `docker exec` command. Here's the step-by-step process:

1. First, find the container ID by running `docker ps`.

2. Once you have the container ID, you can execute the command inside the container using the following command:

```
docker exec -it <container-id> <command>
```

Replace `<container-id>` with the ID of your container, and `<command>` with the command you want to execute. For example, if you want to execute a bash shell, you would use:

```
docker exec -it <container-id> bash
```

If the command you want to execute requires a specific user, you may need to specify it using the `-u` flag, like so:

```
docker exec -it -u <user> <container-id> <command>
```

Replace `<user>` with the username of the user you want to run the command as.

So, in conclusion, to execute a command in a running Docker container, you should use the `docker exec` command followed by the container ID and the command you want to execu